## **ANALYSIS_masters_make_iHS_DH**

### GOALS of this script:
* make the iHS scan within each DH line pool for masters thesis
* calculate for each chromosome seperatly - make sure that calculations dont wrap
* final figure desgin
    * grouped manhattan plot for all chromosomes together?

### **Import packages**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import allel
import random
import h5py
import bcolz
import csv
import itertools
import os
import seaborn as sns; sns.set_theme(color_codes=True)
from collections import Counter
from itertools import combinations
from scipy.stats import uniform
from scipy.stats import randint
from itertools import chain
sns.set_style('white')
sns.set_style('ticks')
import scipy.stats as stats
import math

### **Load the data**

In [ ]:
callset = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf')

In [ ]:
#callset

In [ ]:
sorted(callset.keys())

In [ ]:
IDs=callset['variants/ID']
IDs

In [ ]:
len(IDs)

In [ ]:
positions=callset['variants/POS']
positions

In [ ]:
len(positions)

In [ ]:
#get the samples in the .vcf
samples= callset["samples"]
#samples

In [ ]:
#total number of the samples inn the vcf
len(samples)

In [ ]:
#put all the samples in a Dataframe
sample_subset = pd.DataFrame(data=samples, columns = ['sample_names'])
sample_subset

In [ ]:
for_axis = allel.vcf_to_dataframe('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf', fields= ['variants/ID', 'variants/CHROM', 'variants/POS'])
for_axis

### **Get the names of the landraces from the .vcf file**

#### **Mayer_2020_dhlines** - 941 DH individuals based on three landraces

941 DH individuals based on three landraces - 31 individuals per Pop

In [ ]:
#all Mayer_2020 sample names in the dataframe
mayer_2020_dhlines_samples=sample_subset[sample_subset.sample_names.str.contains("_2020_dh")]
#mayer_2020_dhlines_samples

In [ ]:
#number of all Mayer_2020_dhline individuals
print(len(mayer_2020_dhlines_samples))
#mayer_2020_dhlines_samples

#### *DH_KE*

In [ ]:
#all DH_KE samples in the Mayer_2020_dhlines
df_DH_KE_samples=mayer_2020_dhlines_samples[mayer_2020_dhlines_samples.sample_names.str.contains("DH_KE")]

In [ ]:
df_DH_KE_samples

In [ ]:
#number of all DH_KE individuals
print(len(df_DH_KE_samples))
#df_DH_KE_samples

#### *DH_LL*

In [ ]:
#all DH_LL samples in the Mayer_2020_dhlines
df_DH_LL_samples=mayer_2020_dhlines_samples[mayer_2020_dhlines_samples.sample_names.str.contains("DH_LL")]

In [ ]:
#df_DH_LL_samples

In [ ]:
#number of all DH_LL individuals
print(len(df_DH_LL_samples))
#df_DH_LL_samples

#### *DH_PE*

In [ ]:
#all DH_PE samples in the Mayer_2020_dhlines
df_DH_PE_samples=mayer_2020_dhlines_samples[mayer_2020_dhlines_samples.sample_names.str.contains("DH_PE")]

In [ ]:
df_DH_PE_samples

In [ ]:
#number of all DH_PE individuals 
print(len(df_DH_PE_samples))
#df_DH_PE_samples

In [ ]:
#flatten the DH line lists
df_DH_KE_samples_list=df_DH_KE_samples['sample_names'].values.tolist()
df_DH_LL_samples_list=df_DH_LL_samples['sample_names'].values.tolist()
df_DH_PE_samples_list=df_DH_PE_samples['sample_names'].values.tolist()

#### **Rename the DH lines lists**

In [ ]:
combi_list_KL_DH_landraces=df_DH_KE_samples_list
combi_list_LL_DH_landraces=df_DH_LL_samples_list
combi_list_PE_DH_landraces=df_DH_PE_samples_list

In [ ]:
Counter(callset['variants/CHROM'])

In [ ]:
len(Counter(callset['variants/CHROM']))

#### **Calculate the IHS - DH KL**

In [ ]:
#make IHS lists for KL and KL DH 
list_single_chrom_IHS_KL=[]
list_all_chrom_IHS_KL=[]
IHS_pos_list_KL=[]
IHS_chrom_list_KL=[]
list_ID_KL=[]

In [ ]:
for chrom_KE in range(11):
    #print(chrom)
    chrom_str_KE=str(chrom_KE)
    
    #load the data for DH line
    loc_samples_KL_DH = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf', samples= df_DH_KE_samples_list, region= chrom_str_KE)
    
    #get the genotype data for the DH line
    geno_samples_KL_DH=allel.GenotypeArray(loc_samples_KL_DH['calldata/GT'])
    print(len(geno_samples_KL_DH))
    
    #make allele counts array
    count_samples_KL_DH=geno_samples_KL_DH.count_alleles()
    
    #boolean to check if the SNPs segregate
    is_seg_KL = count_samples_KL_DH.is_segregating()[:]
    
    #apply the seg boolean to the genotype data
    samples_population_KL_is_seg=geno_samples_KL_DH.compress(is_seg_KL, axis=0)
    print(len(samples_population_KL_is_seg))
    
    #make allele counts array form the new genotype array
    count_samples_KL_DH_is_seg=samples_population_KL_is_seg.count_alleles()
    
    #make array of alternate allele counts from the new allele count array
    alt_allele_KL_DH=count_samples_KL_DH_is_seg[:, 1]

    #make haplotype array from the new genotype array
    haplo_samples_KL_DH= samples_population_KL_is_seg.haploidify_samples()
    
    #get the positions for the whole data 
    samples_KL_DH_pos=loc_samples_KL_DH['variants/POS']
    
    #filter the position
    samples_KL_DH_pos_is_seg=samples_KL_DH_pos.compress(is_seg_KL, axis=0)
    
    #get the chromosomes for the whole data
    samples_KL_DH_chrom=loc_samples_KL_DH['variants/CHROM']
    
    #filter the chrom
    samples_KL_DH_chrom_is_seg=samples_KL_DH_chrom.compress(is_seg_KL, axis=0)
    
    #get the IDs for the SNPs
    samples_KL_DH_ID=loc_samples_KL_DH['variants/ID']
    
    #filter the IDs
    samples_KL_DH_ID_is_seg=samples_KL_DH_ID.compress(is_seg_KL, axis=0)
    
    #calculate IHS for KL
    list_single_chrom_IHS_KL_raw=allel.ihs(haplo_samples_KL_DH,samples_KL_DH_pos_is_seg)
    list_single_chrom_IHS_KL_standard, bins =allel.standardize_by_allele_count(list_single_chrom_IHS_KL_raw,alt_allele_KL_DH)
    
    #append elements to IHS lists
    list_all_chrom_IHS_KL.append(list_single_chrom_IHS_KL_standard)
    IHS_pos_list_KL.append(samples_KL_DH_pos_is_seg)
    IHS_chrom_list_KL.append(samples_KL_DH_chrom_is_seg)
    list_ID_KL.append(samples_KL_DH_ID_is_seg)
    
    #print the progress of the calculations 
    print("IHS for DH KL calcualted - %(a)s CHROM" % {'a': str(chrom_KE)})

#### **Calculate the IHS - DH LL**

In [ ]:
#make IHS lists for LL DH 
list_single_chrom_IHS_LL=[]
list_all_chrom_IHS_LL=[]
IHS_pos_list_LL=[]
IHS_chrom_list_LL=[]
list_ID_LL=[]

In [ ]:
for chrom_LL in range(11):
    #print(chrom)
    chrom_str_LL=str(chrom_LL)
    
    #load the data for DH line
    loc_samples_LL_DH = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf', samples= df_DH_LL_samples_list, region= chrom_str_LL)

    #get the genotype data for the DH line
    geno_samples_LL_DH=allel.GenotypeArray(loc_samples_LL_DH['calldata/GT'])
    print(len(geno_samples_LL_DH))
    
    #make allele counts array
    count_samples_LL_DH=geno_samples_LL_DH.count_alleles()
    
    #boolean to check if the SNPs segregate
    is_seg_LL = count_samples_LL_DH.is_segregating()[:]
    
    #apply the seg boolean to the genotype data
    samples_population_LL_is_seg=geno_samples_LL_DH.compress(is_seg_LL, axis=0)
    print(len(samples_population_LL_is_seg))
    
    #make allele counts array form the new genotype array
    count_samples_LL_DH_is_seg=samples_population_LL_is_seg.count_alleles()
    
    #make array of alternate allele counts from the new allele count array
    alt_allele_LL_DH=count_samples_LL_DH_is_seg[:, 1]
    
    #make haplotype array
    haplo_samples_LL_DH= samples_population_LL_is_seg.haploidify_samples()
    
    #get the positions for the whole data 
    samples_LL_DH_pos=loc_samples_LL_DH['variants/POS']
    
    #filter the position
    samples_LL_DH_pos_is_seg=samples_LL_DH_pos.compress(is_seg_LL, axis=0)
    
    #get the chromosomes for the whole data
    samples_LL_DH_chrom=loc_samples_LL_DH['variants/CHROM']
    
    #filter the chrom
    samples_LL_DH_chrom_is_seg=samples_LL_DH_chrom.compress(is_seg_LL, axis=0)
    
    #get the IDs for the SNPs
    samples_LL_DH_ID=loc_samples_LL_DH['variants/ID']
    
    #filter the IDs
    samples_LL_DH_ID_is_seg=samples_LL_DH_ID.compress(is_seg_LL, axis=0)
    
    #calculate IHS for LL
    list_single_chrom_IHS_LL_raw=allel.ihs(haplo_samples_LL_DH,samples_LL_DH_pos_is_seg)
    list_single_chrom_IHS_LL_standard, bins=allel.standardize_by_allele_count(list_single_chrom_IHS_LL_raw,alt_allele_LL_DH)
    
    #append elements to IHS lists
    list_all_chrom_IHS_LL.append(list_single_chrom_IHS_LL_standard)
    IHS_pos_list_LL.append(samples_LL_DH_pos_is_seg)
    IHS_chrom_list_LL.append(samples_LL_DH_chrom_is_seg)
    list_ID_LL.append(samples_LL_DH_ID_is_seg)
    
    #print the progress of the calculations 
    print("IHS for DH LL calcualted - %(a)s CHROM" % {'a': str(chrom_LL)})

#### **Calculate the IHS - DH PE**

In [ ]:
#make xpehh lists for PE and PE DH 
list_single_chrom_IHS_PE=[]
list_all_chrom_IHS_PE=[]
IHS_pos_list_PE=[]
IHS_chrom_list_PE=[]
list_ID_PE=[]

In [ ]:
for chrom_PE in range(11):
    #print(chrom)
    chrom_str_PE=str(chrom_PE)
    
    #load the data for DH line and the corresponding landrace seperatly 
    loc_samples_PE_DH = allel.read_vcf('/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/merge_merge_all_chrom_all_edits_with_allel_column_vcf.vcf', samples= df_DH_PE_samples_list, region= chrom_str_PE)

    #get the genotype data for the 
    geno_samples_PE_DH=allel.GenotypeArray(loc_samples_PE_DH['calldata/GT'])
    print(len(geno_samples_PE_DH))
    
    #make allele counts array
    count_samples_PE_DH=geno_samples_PE_DH.count_alleles()
    
    #boolean to check if the SNPs segregate
    is_seg_PE = count_samples_PE_DH.is_segregating()[:]
    
    #apply the seg boolean to the genotype data
    samples_population_PE_is_seg=geno_samples_PE_DH.compress(is_seg_PE, axis=0)
    print(len(samples_population_PE_is_seg))
    
    #make allele counts array form the new genotype array
    count_samples_PE_DH_is_seg=samples_population_PE_is_seg.count_alleles()
    
    #make array of alternate allele counts from the new allele count array
    alt_allele_PE_DH=count_samples_PE_DH_is_seg[:, 1]
    
    #make haplotype array
    haplo_samples_PE_DH= samples_population_PE_is_seg=samples_population_PE_is_seg.haploidify_samples()
    
    #get the positions for the whole data
    samples_PE_DH_pos=loc_samples_PE_DH['variants/POS']
    
    #filter the position
    samples_PE_DH_pos_is_seg=samples_PE_DH_pos.compress(is_seg_PE, axis=0)

    #get the chromosomes for the whole data
    samples_PE_DH_chrom=loc_samples_PE_DH['variants/CHROM']
    
    #filter the chrom
    samples_PE_DH_chrom_is_seg=samples_PE_DH_chrom.compress(is_seg_PE, axis=0)
    
    #get the IDs for the SNPs
    samples_PE_DH_ID=loc_samples_PE_DH['variants/ID']
    
    #filter the IDs
    samples_PE_DH_ID_is_seg=samples_PE_DH_ID.compress(is_seg_PE, axis=0)
    
    #calculate IHS for PE
    list_single_chrom_IHS_PE_raw=allel.ihs(haplo_samples_PE_DH,samples_PE_DH_pos_is_seg)
    list_single_chrom_IHS_PE_standard, bins =allel.standardize_by_allele_count(list_single_chrom_IHS_PE_raw,alt_allele_PE_DH)
    
    #append elements to xpehh lists 
    list_all_chrom_IHS_PE.append(list_single_chrom_IHS_PE_standard)
    IHS_pos_list_PE.append(samples_PE_DH_pos_is_seg)
    IHS_chrom_list_PE.append(samples_PE_DH_chrom_is_seg)
    list_ID_PE.append(samples_PE_DH_ID_is_seg)
    
    #print the progress of the calculations 
    print("IHS for DH PE calcualted - %(a)s CHROM"% {'a': str(chrom_PE)})

#### **Check the other XPEHH lists**

**Check the list legnths based on chrom lengths**

In [ ]:
Counter(callset['variants/CHROM'])

*KL & KL DH*

In [ ]:
for KL_chrom in list_all_chrom_IHS_KL:
    print(len(KL_chrom))

In [ ]:
for KL_pos in IHS_pos_list_KL:
    print(len(KL_pos))

In [ ]:
for KL_chrom_num in IHS_chrom_list_KL:
    print(len(KL_chrom_num))

*LL & LL DH*

In [ ]:
for LL_chrom in list_all_chrom_IHS_LL:
    print(len(LL_chrom))

In [ ]:
for LL_pos in IHS_pos_list_LL:
    print(len(LL_pos))

In [ ]:
for LL_chrom_num in IHS_chrom_list_LL:
    print(len(LL_chrom_num))

*PE & PE DH*

In [ ]:
for PE_chrom in list_all_chrom_IHS_PE:
    print(len(PE_chrom))

In [ ]:
for PE_pos in IHS_pos_list_PE:
    print(len(PE_pos))

In [ ]:
for PE_chrom_num in IHS_chrom_list_PE:
    print(len(PE_chrom_num))

#### **Make density plot for each DH population**

**Flatten the IHS vlaue lists**

In [ ]:
list_all_chrom_IHS_KL_flatten=list(itertools.chain(*list_all_chrom_IHS_KL))

In [ ]:
list_all_chrom_IHS_LL_flatten=list(itertools.chain(*list_all_chrom_IHS_LL))

In [ ]:
list_all_chrom_IHS_PE_flatten=list(itertools.chain(*list_all_chrom_IHS_PE))

In [ ]:
len(list_all_chrom_IHS_KL_flatten)

In [ ]:
len(list_all_chrom_IHS_LL_flatten)

In [ ]:
len(list_all_chrom_IHS_PE_flatten)

**Make the Gaussian Distributions**

In [ ]:
#normal distribution
mu = 0 #mu is the mean or expectation, location of max 
variance = 1 #variance is standard deviation
sigma = math.sqrt(variance) #sigma is standard deviation squared
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100) #make a normal distribution with 100 values 
plt.plot(x, stats.norm.pdf(x, mu, sigma)) # plot the normal distribution 

#set the plot parameters 
plt.xlim(-5,5) #x-axis limits 
plt.ylim(0,0.45) #y-axis limits 
sns.despine(offset={'bottom': 20}) #move the x-axis away
plt.show()

**Plot the IHS density plot & Gaussian distribution**

In [ ]:
#KL IHS
sns.kdeplot(data=list_all_chrom_IHS_KL_flatten).set_title("Genome-wide distribution standardized IHS - DH KL") #plot IHS
plt.plot(x, stats.norm.pdf(x, mu, sigma)) #plot normal distribution

#set the plot parameters
plt.xlim(-5,5) #x-limit axis
plt.ylim(0,0.45) #y-limit axis
sns.despine(offset={'bottom': 20}) #move the x-axis away
plt.show()

In [ ]:
#LL IHS
sns.kdeplot(list_all_chrom_IHS_LL_flatten).set_title("Genome-wide distribution standardized IHS - DH LL") #plot IHS
plt.plot(x, stats.norm.pdf(x, mu, sigma)) #plot normal distribution

#set the plot parameters
plt.xlim(-5,5) #x-limit axis
plt.ylim(0,0.45) #y-limit axis 
sns.despine(offset={'bottom': 20}) #move the x-axis away
plt.show()

In [ ]:
#PE IHS
sns.kdeplot(list_all_chrom_IHS_PE_flatten).set_title("Genome-wide distribution standardized IHS - DH PE") #plot IHS
plt.plot(x, stats.norm.pdf(x, mu, sigma)) #plot normal distribution

#set the plot parameters
plt.xlim(-5,5) #x-limit axis
plt.ylim(0,0.45) #y-limit axis 
sns.despine(offset={'bottom': 20}) # move the x-axis aways
plt.show()

#### **Make a IHS dataframe**

**Flatten the lists for the dataframes** - sum(l, []) should also work

*KL DH*

In [ ]:
IHS_pos_list_KL_flatten=list(itertools.chain(*IHS_pos_list_KL))

In [ ]:
IHS_chrom_list_KL_flatten=list(itertools.chain(*IHS_chrom_list_KL))

In [ ]:
IHS_ID_list_KL_flatten=list(itertools.chain(*list_ID_KL))

In [ ]:
#check the lengths 
print(len(list_all_chrom_IHS_KL_flatten))
print(len(IHS_pos_list_KL_flatten))
print(len(IHS_chrom_list_KL_flatten))
print(len(IHS_ID_list_KL_flatten))

In [ ]:
range_len_KL=len(list_all_chrom_IHS_KL_flatten)

In [ ]:
#make the dataframe
KL_dataframe = pd.DataFrame(
    {'ID_KL':IHS_ID_list_KL_flatten,
     'chrom_KL': IHS_chrom_list_KL_flatten,
     'pos_KL': IHS_pos_list_KL_flatten,
     'IHS_KL': list_all_chrom_IHS_KL_flatten,
     'ind': range(len(range(range_len_KL)))
    })

In [ ]:
KL_dataframe['chrom_KL'] = 'Chr ' + KL_dataframe['chrom_KL'].astype(str)

In [ ]:
KL_dataframe

*LL DH*

In [ ]:
IHS_pos_list_LL_flatten=list(itertools.chain(*IHS_pos_list_LL))

In [ ]:
IHS_chrom_list_LL_flatten=list(itertools.chain(*IHS_chrom_list_LL))

In [ ]:
IHS_ID_list_LL_flatten=list(itertools.chain(*list_ID_LL))

In [ ]:
#check the lengths 
print(len(list_all_chrom_IHS_LL_flatten))
print(len(IHS_pos_list_LL_flatten))
print(len(IHS_chrom_list_LL_flatten))
print(len(IHS_ID_list_LL_flatten))

In [ ]:
range_len_LL=len(list_all_chrom_IHS_LL_flatten)

In [ ]:
#make the dataframe 
LL_dataframe = pd.DataFrame(
    {'ID_LL':IHS_ID_list_LL_flatten,
     'chrom_LL': IHS_chrom_list_LL_flatten,
     'pos_LL': IHS_pos_list_LL_flatten,
     'IHS_LL': list_all_chrom_IHS_LL_flatten,
     'ind': range(len(range(range_len_LL)))
    })

In [ ]:
LL_dataframe['chrom_LL'] = 'Chr ' + LL_dataframe['chrom_LL'].astype(str)

In [ ]:
LL_dataframe

*PE DH*

In [ ]:
IHS_pos_list_PE_flatten=list(itertools.chain(*IHS_pos_list_PE))

In [ ]:
IHS_chrom_list_PE_flatten=list(itertools.chain(*IHS_chrom_list_PE))

In [ ]:
IHS_ID_list_PE_flatten=list(itertools.chain(*list_ID_PE))

In [ ]:
#check the lengths 
print(len(list_all_chrom_IHS_PE_flatten))
print(len(IHS_pos_list_PE_flatten))
print(len(IHS_chrom_list_PE_flatten))
print(len(IHS_ID_list_PE_flatten))

In [ ]:
range_len_PE=len(list_all_chrom_IHS_PE_flatten)

In [ ]:
#make the dataframe 
PE_dataframe = pd.DataFrame(
    {'ID_PE':IHS_ID_list_PE_flatten,
     'chrom_PE': IHS_chrom_list_PE_flatten,
     'pos_PE': IHS_pos_list_PE_flatten,
     'IHS_PE': list_all_chrom_IHS_PE_flatten,
     'ind': range(len(range(range_len_PE)))
    })

In [ ]:
PE_dataframe['chrom_PE'] = 'Chr ' + PE_dataframe['chrom_PE'].astype(str)

In [ ]:
PE_dataframe

#### **Calculate p-value for each IHS value**

piHS=−log10(2Φ(−|iHS|))

In [ ]:
KL_dataframe['pIHS_KL']=-np.log10(2 * stats.norm.cdf(-abs(KL_dataframe.IHS_KL)))

In [ ]:
LL_dataframe['pIHS_LL']=-np.log10(2 * stats.norm.cdf(-abs(LL_dataframe.IHS_LL)))

In [ ]:
PE_dataframe['pIHS_PE']=-np.log10(2 * stats.norm.cdf(-abs(PE_dataframe.IHS_PE)))

#### **Make Manhattan Plot**

In [ ]:
for_axis

In [ ]:
KL_dataframe_re=KL_dataframe.rename(columns={"ID_KL": "ID", "chrom_KL":"CHROM" , "pos_KL":"POS"})

In [ ]:
#KL_dataframe_re

In [ ]:
KL_dataframe_re_dropped=KL_dataframe_re.drop(columns=["ind","IHS_KL","CHROM"])
KL_dataframe_re_dropped

In [ ]:
LL_dataframe_re=LL_dataframe.rename(columns={"ID_LL": "ID", "chrom_LL":"CHROM" , "pos_LL":"POS"})

In [ ]:
#LL_dataframe_re

In [ ]:
LL_dataframe_re_dropped=LL_dataframe_re.drop(columns=["ind","IHS_LL","CHROM"])
LL_dataframe_re_dropped

In [ ]:
PE_dataframe_re=PE_dataframe.rename(columns={"ID_PE": "ID", "chrom_PE":"CHROM" , "pos_PE":"POS"})

In [ ]:
#PE_dataframe_re

In [ ]:
PE_dataframe_re_dropped=PE_dataframe_re.drop(columns=["ind","IHS_PE","CHROM"])
PE_dataframe_re_dropped

In [ ]:
merge_1 = pd.merge(for_axis,KL_dataframe_re_dropped, how="outer", on=["ID","POS"])

In [ ]:
merge_1

In [ ]:
merge_2 = pd.merge(for_axis,LL_dataframe_re_dropped, how="outer", on=["ID","POS"])

In [ ]:
merge_2

In [ ]:
merge_3 = pd.merge(for_axis,PE_dataframe_re_dropped, how="outer", on=["ID","POS"])

In [ ]:
merge_3

In [ ]:
merge_all=merge_1.merge(merge_2,on=["ID","POS","CHROM"]).merge(merge_3,on=["ID","POS","CHROM"])
merge_all

In [ ]:
merge_all.loc[merge_all['pIHS_PE'] > 6.9]



In [ ]:
result_1=merge_all.loc[merge_all['CHROM'] == "1"]
result_1

In [ ]:
result_2=merge_all.loc[merge_all['CHROM'] == "2"]
result_2

In [ ]:
result_3=merge_all.loc[merge_all['CHROM'] == "3"]
result_3

In [ ]:
result_4=merge_all.loc[merge_all['CHROM'] == "4"]
result_4

In [ ]:
result_5=merge_all.loc[merge_all['CHROM'] == "5"]
result_5

In [ ]:
result_6=merge_all.loc[merge_all['CHROM'] == "6"]
result_6

In [ ]:
result_7=merge_all.loc[merge_all['CHROM'] == "7"]
result_7

In [ ]:
result_8=merge_all.loc[merge_all['CHROM'] == "8"]
result_8

In [ ]:
result_9=merge_all.loc[merge_all['CHROM'] == "9"]
result_9

In [ ]:
result_10=merge_all.loc[merge_all['CHROM'] == "10"]
result_10

In [ ]:
result_list=["result_1","result_2","result_3","result_4","result_5","result_6","result_7","result_8","result_9","result_10"]

In [ ]:
pal=sns.color_palette("hls", 10)
hex_col=pal.as_hex()
print(hex_col)

In [ ]:
for i in range(10):
    #print(i)
    num=i+1
    #print(num)
    data_select=merge_all.loc[merge_all['CHROM'] == str(num)] 
    
    fig1, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1)
    
    fig1.set_size_inches(10, 15)
    fig1.subplots_adjust(hspace=0.2)
    
    sns.scatterplot(data=data_select, x="POS", y="pIHS_KL", ax=ax1, color=hex_col[i])
    ax1.set(ylim=(0,17))
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.set_ylabel("-log10(pIHS value)",fontweight="bold")
    ax1.set_xlabel("Chr " + str(num) + " - pIHS_KL",fontweight="bold",fontsize=10)
    ax1.axhline(y=6.99, color='black', linestyle='dashed',linewidth=1)

    sns.scatterplot(data=data_select, x="POS", y="pIHS_LL", ax=ax2, color=hex_col[i])
    ax2.set(ylim=(0,17))
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.set_ylabel("-log10(pIHS value)",fontweight="bold")
    ax2.set_xlabel("Chr " + str(num) + " - pIHS_LL",fontweight="bold",fontsize=10)
    ax2.axhline(y=6.99, color='black', linestyle='dashed',linewidth=1)

    sns.scatterplot(data=data_select, x="POS", y="pIHS_PE", ax=ax3, color=hex_col[i])
    ax3.set(ylim=(0,17))
    ax3.spines['top'].set_visible(False)
    ax3.spines['right'].set_visible(False)
    ax3.set_ylabel("-log10(pIHS value)",fontweight="bold")
    ax3.set_xlabel("Chr " + str(num) + " - pIHS_PE",fontweight="bold",fontsize=10)
    ax3.axhline(y=6.99, color='black', linestyle='dashed',linewidth=1)
    
    plt.savefig('piHS'+str(num)+'.png')
    
    plt.clf()
    